1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
import pandas as pd

Наши новости

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2022-06-28 06:36:25--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 172.253.62.102, 172.253.62.139, 172.253.62.113, ...
Connecting to docs.google.com (docs.google.com)|172.253.62.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mm9j3avo6j0oktug474bnqo0rkl7gq5a/1656398175000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download [following]
--2022-06-28 06:36:25--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mm9j3avo6j0oktug474bnqo0rkl7gq5a/1656398175000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 172.253.62.132, 2607:f8b0:4004:c07::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0

In [ ]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [ ]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [ ]:
!pip install razdel pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 8.2 MB 42.9 MB/s 


In [ ]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2022-06-28 06:36:41--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 172.253.63.102, 172.253.63.138, 172.253.63.113, ...
Connecting to drive.google.com (drive.google.com)|172.253.63.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vf6o58va2428oq1ufsa57lv75n3f87nb/1656398175000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download [following]
--2022-06-28 06:36:41--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vf6o58va2428oq1ufsa57lv75n3f87nb/1656398175000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 172.253.62.132, 2607:f8b0:4004:c07::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs

In [ ]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

stopword_ru += additional_stopwords
len(stopword_ru)

776

In [ ]:
def clean_text(text):
    '''
    очистка текста

    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)

    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]

    return words_lem_without_stopwords

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/27000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Possible nested set at position 39
  
100%|██████████| 27000/27000 [00:47<00:00, 565.72it/s] 


In [ ]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [ ]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [04:35<00:00, 98.03it/s] 


А теперь в 3 строчки обучим нашу модель

In [ ]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [ ]:
N_topic = 20  # можно было менять

In [ ]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [ ]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [ ]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [ ]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(0, 0.077043995), (1, 0.22554448), (5, 0.5655469), (6, 0.11186461)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [ ]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: украина ракета млрд гражданин украинский ставка рост
topic_1: россия развитие правительство рубль москва население проект
topic_2: журнал век женщина собака мужчина немец свет
topic_3: обнаружить земля день исследователь животное место тело
topic_4: самолёт корабль полёт космический аппарат поверхность пенсия
topic_5: автор журнал фестиваль сайт узнать участник налог
topic_6: всё научный очень газ первый день жизнь
topic_7: погибнуть северный южный исследователь земля океан японский
topic_8: взрыв сирия жертва теракт террорист баррель инцидент
topic_9: смерть путин убийство квартира россия владимир следователь
topic_10: банк рубль ребёнок тыс руб лицо деньга
topic_11: система экономика новый экономический спутник китай первый
topic_12: рак авиакомпания ген выяснить белый исследователь военный
topic_13: россия военный российский проект эксперт наука новый
topic_14: фонд выяснить газпром px воздух сколково сектор
topic_15: мозг млн тыс первый составить опубликовать научный
topic

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [ ]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [ ]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.30736628, 0.        , 0.        , 0.        ,
       0.0711675 , 0.01331169, 0.        , 0.        , 0.05829797,
       0.        , 0.        , 0.        , 0.2539998 , 0.23017488,
       0.        , 0.        , 0.        , 0.05998013, 0.        ])

In [ ]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.000000,0.307368,0.000000,0.0,0.0,0.071192,0.013265,0.0,0.000000,...,0.0,0.000000,0.0,0.254012,0.230194,0.000000,0.0,0.0,0.059935,0.0
1,4896,0.000000,0.087506,0.035486,0.0,0.0,0.393137,0.000000,0.0,0.464358,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,4897,0.077501,0.225344,0.000000,0.0,0.0,0.565956,0.111199,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,4898,0.000000,0.000000,0.000000,0.0,0.0,0.136324,0.440786,0.0,0.141407,...,0.0,0.025066,0.0,0.212476,0.000000,0.035405,0.0,0.0,0.000000,0.0
4,4899,0.000000,0.000000,0.000000,0.0,0.0,0.119628,0.000000,0.0,0.000000,...,0.0,0.243513,0.0,0.428364,0.000000,0.000000,0.0,0.0,0.186873,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [ ]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [ ]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [ ]:
doc_dict[293672]

array([0.        , 0.05779359, 0.        , 0.33091781, 0.        ,
       0.08105773, 0.        , 0.        , 0.        , 0.03115397,
       0.        , 0.02599074, 0.        , 0.36630619, 0.        ,
       0.        , 0.        , 0.        , 0.06495859, 0.02982141])

In [ ]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [ ]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.01899095, 0.11890423, 0.        , 0.01196294, 0.05339328,
       0.00216234, 0.047694  , 0.00177318, 0.0063457 , 0.07665569,
       0.15284712, 0.04728873, 0.03647327, 0.11440088, 0.        ,
       0.00823157, 0.10849911, 0.10767383, 0.02619256, 0.05008863])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [ ]:
FUNC = np.mean

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.062383,0.032015,0.019581,0.124099,0.000000,0.123069,0.049080,0.005389,0.069620,...,0.116120,0.008152,0.002730,0.065834,0.000000,0.087963,0.029354,0.044953,0.034981,0.086272
1,u108690,0.092629,0.147461,0.002670,0.025746,0.002324,0.014994,0.120358,0.000000,0.038523,...,0.137084,0.018121,0.002736,0.037017,0.009057,0.003888,0.067187,0.080557,0.053537,0.060971
2,u108339,0.016160,0.045955,0.000000,0.133936,0.018030,0.002517,0.043330,0.011321,0.044193,...,0.146098,0.029479,0.005683,0.062505,0.016418,0.031856,0.104355,0.043822,0.111926,0.063199


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2022-06-28 06:44:14--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 172.253.63.139, 172.253.63.113, 172.253.63.100, ...
Connecting to drive.google.com (drive.google.com)|172.253.63.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/it0r4sm2tdt4h1solqr0lkidmu8j2q4o/1656398625000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download [following]
--2022-06-28 06:44:14--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/it0r4sm2tdt4h1solqr0lkidmu8j2q4o/1656398625000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 172.253.62.132, 2607:f8b0:4004:c07::84
Connecting to doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.062383,0.032015,0.019581,0.124099,0.000000,0.123069,0.049080,0.005389,0.069620,...,0.008152,0.002730,0.065834,0.000000,0.087963,0.029354,0.044953,0.034981,0.086272,0
1,u108690,0.092629,0.147461,0.002670,0.025746,0.002324,0.014994,0.120358,0.000000,0.038523,...,0.018121,0.002736,0.037017,0.009057,0.003888,0.067187,0.080557,0.053537,0.060971,1
2,u108339,0.016160,0.045955,0.000000,0.133936,0.018030,0.002517,0.043330,0.011321,0.044193,...,0.029479,0.005683,0.062505,0.016418,0.031856,0.104355,0.043822,0.111926,0.063199,1


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.11456499, 0.01262251, 0.45878247, 0.44986422, 0.08569991,
       0.11503506, 0.0867749 , 0.09542294, 0.06613615, 0.11812645])

In [ ]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [ ]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.30801093292501325, F-Score=0.713, Precision=0.716, Recall=0.710


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9553020524449095

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.308011,0.713115,0.716049,0.710204,0.955302


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [ ]:
FUNC = np.median

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.028897,0.0,0.103240,0.000000,0.079788,0.038024,0.0,0.000000,...,0.074614,0.000000,0.0,0.000000,0.000000,0.028464,0.000000,0.00000,0.000000,0.055838
1,u108690,0.042314,0.125097,0.0,0.028256,0.000000,0.000000,0.072431,0.0,0.029335,...,0.115434,0.000000,0.0,0.039934,0.000000,0.000000,0.054927,0.00544,0.006715,0.036430
2,u108339,0.008538,0.033739,0.0,0.065436,0.006915,0.000000,0.048141,0.0,0.017348,...,0.146438,0.026288,0.0,0.023761,0.009945,0.006313,0.000000,0.02695,0.124190,0.056246


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.028897,0.0,0.103240,0.000000,0.079788,0.038024,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.028464,0.000000,0.00000,0.000000,0.055838,0
1,u108690,0.042314,0.125097,0.0,0.028256,0.000000,0.000000,0.072431,0.0,0.029335,...,0.000000,0.0,0.039934,0.000000,0.000000,0.054927,0.00544,0.006715,0.036430,1
2,u108339,0.008538,0.033739,0.0,0.065436,0.006915,0.000000,0.048141,0.0,0.017348,...,0.026288,0.0,0.023761,0.009945,0.006313,0.000000,0.02695,0.124190,0.056246,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.08999594, 0.00713153, 0.4813815 , 0.40798516, 0.08347724,
       0.06071377, 0.0928759 , 0.16293123, 0.04915171, 0.16613185])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.302898854182675, F-Score=0.752, Precision=0.769, Recall=0.735


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9645328216756787

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.308011,0.713115,0.716049,0.710204,0.955302
1,median,0.302899,0.751566,0.769231,0.734694,0.964533



3. Повторить п.2, но используя уже не медиану, а max


In [ ]:
FUNC = np.max

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.232423,0.069888,0.117489,0.330918,0.000000,0.340741,0.122889,0.032334,0.231983,...,0.276339,0.025991,0.016381,0.366306,0.000000,0.260305,0.128184,0.247024,0.144927,0.300353
1,u108690,0.296334,0.287015,0.016021,0.055022,0.013944,0.048941,0.308129,0.000000,0.104511,...,0.402543,0.083770,0.016419,0.084657,0.054344,0.023330,0.160590,0.409286,0.254788,0.147057
2,u108339,0.044127,0.112746,0.000000,0.347865,0.068541,0.015101,0.073907,0.036893,0.158408,...,0.298438,0.072704,0.034097,0.174184,0.055585,0.152593,0.393400,0.159686,0.230807,0.171170


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.232423,0.069888,0.117489,0.330918,0.000000,0.340741,0.122889,0.032334,0.231983,...,0.025991,0.016381,0.366306,0.000000,0.260305,0.128184,0.247024,0.144927,0.300353,0
1,u108690,0.296334,0.287015,0.016021,0.055022,0.013944,0.048941,0.308129,0.000000,0.104511,...,0.083770,0.016419,0.084657,0.054344,0.023330,0.160590,0.409286,0.254788,0.147057,1
2,u108339,0.044127,0.112746,0.000000,0.347865,0.068541,0.015101,0.073907,0.036893,0.158408,...,0.072704,0.034097,0.174184,0.055585,0.152593,0.393400,0.159686,0.230807,0.171170,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.06605422, 0.0022967 , 0.76341394, 0.34511662, 0.06969475,
       0.0356669 , 0.00780046, 0.01616019, 0.1377724 , 0.08239497])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3214999103813633, F-Score=0.745, Precision=0.685, Recall=0.816


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9592022792022791

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.308011,0.713115,0.716049,0.710204,0.955302
1,median,0.302899,0.751566,0.769231,0.734694,0.964533
2,amax,0.321500,0.744879,0.684932,0.816327,0.959202


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [ ]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [ ]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)

    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [ ]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|██████████| 8000/8000 [01:23<00:00, 95.62it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.248542,0.0,0.856067,0.000000,0.686257,0.311631,0.0,0.000000,...,0.688781,0.000000,0.0,0.000000,0.000000,0.233276,0.000000,0.000000,0.000000,0.480262
1,u108690,0.357192,1.127399,0.0,0.235039,0.000000,0.000000,0.606638,0.0,0.242941,...,0.975691,0.000000,0.0,0.342625,0.000000,0.000000,0.488229,0.050556,0.055030,0.312546
2,u108339,0.073433,0.286854,0.0,0.532726,0.057935,0.000000,0.399368,0.0,0.161231,...,1.244014,0.220237,0.0,0.204373,0.083316,0.054296,0.000000,0.219260,1.104218,0.497698


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.248542,0.0,0.856067,0.000000,0.686257,0.311631,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.233276,0.000000,0.000000,0.000000,0.480262,0
1,u108690,0.357192,1.127399,0.0,0.235039,0.000000,0.000000,0.606638,0.0,0.242941,...,0.000000,0.0,0.342625,0.000000,0.000000,0.488229,0.050556,0.055030,0.312546,1
2,u108339,0.073433,0.286854,0.0,0.532726,0.057935,0.000000,0.399368,0.0,0.161231,...,0.220237,0.0,0.204373,0.083316,0.054296,0.000000,0.219260,1.104218,0.497698,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.03495514e-02, 9.43409410e-08, 8.92337328e-01, 4.24744074e-01,
       1.05871340e-02, 1.96594997e-03, 2.11511495e-02, 3.42138683e-02,
       2.26457444e-03, 5.86386597e-02])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4719134284040988, F-Score=0.834, Precision=0.848, Recall=0.820


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9859015059015058

In [ ]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)


5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [ ]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.308011,0.713115,0.716049,0.710204,0.955302
1,median,0.302899,0.751566,0.769231,0.734694,0.964533
2,amax,0.321500,0.744879,0.684932,0.816327,0.959202
3,idf_median,0.471913,0.834025,0.848101,0.820408,0.985902


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных